# Sampling pose DKF trained on H3.6M

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import addpaths
from load import loadDataset
from h36m_loader import insert_junk_entries
import os
import numpy as np
from scipy.signal import convolve2d

# Stupid hack to make parameter loading actually work
# (ugh, this is super brittle code)
import sys
del sys.argv[1:]

# CONFIG_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-R-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-uid-config.pkl'
# WEIGHT_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-R-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-uid-EP375-params.npz'
# sys.argv.extend('-vm R -infm structured -ds 10 -dh 50'.split())

# CONFIG_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-L-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-past-only-config.pkl'
# WEIGHT_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-L-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-past-only-EP25-params.npz'
# sys.argv.extend('-vm L -infm structured -ds 10 -dh 50 -uid past-only'.split())

# CONFIG_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-L-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-conditional-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-past-only-cond-emis-config.pkl'
# WEIGHT_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-L-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-conditional-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-past-only-cond-emis-EP25-params.npz'
# sys.argv.extend('-vm L -infm structured -ds 10 -dh 50 -uid past-only-cond-emis -etype conditional'.split())

CONFIG_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-L-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-cond-True-past-only-config.pkl'
WEIGHT_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-L-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-cond-True-past-all-act-cond-EP1075-params.npz'
sys.argv.extend('-vm L -cond -infm structured -ds 10 -dh 50 -uid past-only'.split())

sys.argv.extend(['-reload', WEIGHT_PATH, '-params', CONFIG_PATH])

In [ ]:
dataset = loadDataset(use_cond=True)

In [ ]:
from parse_args_dkf import parse; params = parse()
from utils.misc import removeIfExists,createIfAbsent,mapPrint,saveHDF5,displayTime
from stinfmodel_fast.dkf import DKF
import stinfmodel_fast.learning as DKF_learn
import stinfmodel_fast.evaluate as DKF_evaluate

In [ ]:
if 'h36m_action_names' in dataset:
    act_names = dataset['h36m_action_names']
    print('Action names: ' + ', '.join(map(str, act_names)))
    one_hot_acts = {}
    hot_vec_size = len(act_names)
    for hot_bit, name in enumerate(act_names):
        one_hot_acts[name] = (np.arange(hot_vec_size) == hot_bit)
else:
    print('No action names found')

In [ ]:
use_cond = bool(params.get('use_cond', False))
params['savedir']+='-h36m'
# createIfAbsent(params['savedir'])

# Add dataset and NADE parameters to "params" which will become part of the
# model
for k in ['dim_observations','data_type']:
    params[k] = dataset[k]
mapPrint('Options: ',params)
if params['use_nade']:
    params['data_type']='real_nade'

# Remove from params
removeIfExists('./NOSUCHFILE')
reloadFile = params.pop('reloadFile')
pfile=params.pop('paramFile')
# paramFile is set inside the BaseClass in theanomodels
# to point to the pickle file containing params"""
assert os.path.exists(pfile),pfile+' not found. Need paramfile'
print 'Reloading trained model from : ',reloadFile
print 'Assuming ',pfile,' corresponds to model'
dkf  = DKF(params, paramFile = pfile, reloadFile = reloadFile)

In [ ]:
def smooth_seq(seq):
    assert seq.ndim == 2, "need 2d seq (real shape %r)" % (seq,)
    kernel = [0.1, 0.25, 0.3, 0.25, 0.1]
    full_kernel = np.array(kernel).reshape((-1, 1))
    rv = convolve2d(seq, full_kernel, mode='valid')
    assert rv.ndim == 2
    assert rv.shape[1] == seq.shape[1]
    assert rv.shape[0] <= seq.shape[0]
    return rv

In [ ]:
if not use_cond:
    # No need to do conditional nonsense!
    oodles_of_samples = dkf.sample(nsamples=50, T=1024)
    sample_X, sample_Z = oodles_of_samples

    print('Output shape: %s' % str(sample_X.shape))
    mu = dataset['h36m_mean'].reshape((1, 1, -1))
    sigma = dataset['h36m_std'].reshape((1, 1, -1))
    real_X = insert_junk_entries(sample_X * sigma + mu)
    dest_dir = './generated/'
    try:
        os.makedirs(dest_dir)
    except OSError:
        pass
    for i, sampled_times in enumerate(real_X):
        dest_fn = os.path.join(dest_dir, 'seq-%i.txt' % i)
        print('Saving %s' % dest_fn)
        np.savetxt(dest_fn, sampled_times, delimiter=',', fmt='%f')

    # Do the same thing, but smoothed
    smooth_dest_dir = './generated-smooth/'
    try:
        os.makedirs(smooth_dest_dir)
    except OSError:
        pass
    for i, sampled_times in enumerate(real_X):
        dest_fn = os.path.join(smooth_dest_dir, 'seq-%i.txt' % i)
        print('Saving %s' % dest_fn)
        smooth_times = smooth_seq(sampled_times)
        np.savetxt(dest_fn, smooth_times, delimiter=',', fmt='%f')

In [ ]:
if use_cond:
    seqs_per_act = 2
    seq_length = 256
    dest_dir = './generated-wacts/'
    try:
        os.makedirs(dest_dir)
    except OSError:
        pass
        
    # start by generating some sequences for each action type
    for act_name, one_hot_rep in one_hot_acts.items():
        print('Working on action %s' % act_name)
        U = np.stack([one_hot_rep] * seq_length, axis=0)
        oodles_of_samples = dkf.sample(nsamples=seqs_per_act, T=seq_length, U=U)
        sample_X, sample_Z = oodles_of_samples
        mu = dataset['h36m_mean'].reshape((1, 1, -1))
        sigma = dataset['h36m_std'].reshape((1, 1, -1))
        real_X = insert_junk_entries(sample_X * sigma + mu)

        for i, sampled_times in enumerate(real_X):
            dest_pfx = os.path.join(dest_dir, 'act-%s-seq-%i' % (act_name, i))
            dest_fn = dest_pfx + '.txt'
            print('Saving ' + dest_fn)
            np.savetxt(dest_fn, sampled_times, delimiter=',', fmt='%f')
            
            dest_fn_smooth = dest_pfx + '-smooth.txt'
            print('Saving ' + dest_fn_smooth)
            smooth_sampled_times = smooth_seq(sampled_times)
            np.savetxt(dest_fn_smooth, smooth_sampled_times, delimiter=',', fmt='%f')

    # now choose random pairs of (distinct) actions and simulate
    # a transition at half-way point
    num_pairs = 10
    nacts = len(act_names)
    chosen_idxs = np.random.permutation(nacts * (nacts-1))[:num_pairs]
    act_pairs = [(act_names[idxp%nacts], act_names[idxp//nacts]) \
                 for idxp in chosen_idxs]
    # act_pairs = [('walking', 'eating'), ('eating', 'walking'),
    #              ('walking', 'smoking'), ('smoking', 'walking'),
    #              ('smoking', 'eating'), ('eating', 'smoking')]
    for act1, act2 in act_pairs:
        print('Computing sequence for action %s -> %s' % (act1, act2))
        
        len1 = seq_length // 2
        len2 = seq_length - len1
        rep1 = one_hot_acts[act1]
        rep2 = one_hot_acts[act2]
        U = np.stack([rep1] * len1 + [rep2] * len2, axis=0)
        oodles_of_samples = dkf.sample(nsamples=seqs_per_act, T=seq_length, U=U)
        sample_X, sample_Z = oodles_of_samples
        mu = dataset['h36m_mean'].reshape((1, 1, -1))
        sigma = dataset['h36m_std'].reshape((1, 1, -1))
        real_X = insert_junk_entries(sample_X * sigma + mu)
        
        for i, sampled_times in enumerate(real_X):
            dest_pfx = os.path.join(dest_dir, 'trans-%s-to-%s-seq-%i' % (act1, act2, i))
            dest_fn = dest_pfx + '.txt'
            print('Saving ' + dest_fn)
            np.savetxt(dest_fn, sampled_times, delimiter=',', fmt='%f')
            
            dest_fn_smooth = dest_pfx + '-smooth.txt'
            print('Saving ' + dest_fn_smooth)
            smooth_sampled_times = smooth_seq(sampled_times)
            np.savetxt(dest_fn_smooth, smooth_sampled_times, delimiter=',', fmt='%f')